In [1]:
import openpyxl
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn.preprocessing
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets.samples_generator import make_blobs

/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


# Acquire Data
I had to do it a special way since it us a a .xlsx

In [2]:
wb = openpyxl.load_workbook("vgsales.csv.xlsx")
ws = wb.active

df = pd.DataFrame(list(ws.iter_rows(values_only=True)))

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
1,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33


# Prepare Data

In [4]:
# Set the column names 
    # had to do this because column names became first row
df.columns = df.iloc[0]

In [5]:
df.sample()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
3670,3671,Soccer Tsuku 2002: J-League Pro Soccer Club o ...,PS2,2002,Sports,Sega,0,0,0.55,0,0.55


In [6]:
# set the index of the dataframe
df = df.set_index('Name')

In [7]:
df.sample()

,Rank,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Name,,,,,,,,,,
LEGO Rock Raiders,10658,PS,1999,Strategy,LEGO Media,0.06,0.04,0,0.01,0.1


In [ ]:
df.head()

In [ ]:
df.Year.value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# drop N/A values
df = df[~df['Year'].isin(['N/A'])]

In [ ]:
# Change data types
df['Year'] = df['Year'].astype(int)
df['Global_Sales'] = df['Global_Sales'].astype(float)

In [ ]:
# dropping these because they feed right into our target and Name is not needed
df = df.drop(['NA_Sales', 'EU_Sales', 
             'JP_Sales', 'Other_Sales', 'Name'], axis=1)


In [ ]:
df.Year.max()

In [ ]:
df['years_binned'] = pd.cut(df.Year, 
                            bins = [1980,1985,1990,1995,2000,2005,2010,2015,2020],
                            labels = ['80-85', '85-90', '90-95', "95-00's", 
                                      "00's-05", '05-10', '10-15', '15-20'])

In [ ]:
df.Global_Sales.max()

In [ ]:
df.Global_Sales.unique()

In [ ]:
# bin the year column to help exploration
df['level_of_success'] = pd.cut(df.Global_Sales, 
                        bins = [0,0.1,0.5,0.9,100],
                        labels = ['Moderate Success', 'Fairly Successful', 'Very Successful', 
                                  'Extremely Successful'])

In [ ]:
df.head()

In [ ]:
df.Global_Sales.max()

Takeaways:
- There was a HUGE spike in sales in the 2000's and continued to grow until there was a steady decline starting near 2010.

df.Platform.value_counts()

In [ ]:
# Sony
df['Platform'] = df.Platform.replace('PS2','Playstation')
df['Platform'] = df.Platform.replace('PS3','Playstation')
df['Platform'] = df.Platform.replace('PSP','Playstation')
df['Platform'] = df.Platform.replace('PS','Playstation')
df['Platform'] = df.Platform.replace('PS4','Playstation')
# Xbox
df['Platform'] = df.Platform.replace('X360','Xbox')
df['Platform'] = df.Platform.replace('XB','Xbox')
df['Platform'] = df.Platform.replace('XOne','Xbox')
# Nintendo
df['Platform'] = df.Platform.replace('DS','Nintendo')
df['Platform'] = df.Platform.replace('Wii','Nintendo')
df['Platform'] = df.Platform.replace('GB','Nintendo')
df['Platform'] = df.Platform.replace('GC','Nintendo')
df['Platform'] = df.Platform.replace('3DS','Nintendo')
df['Platform'] = df.Platform.replace('SNES','Nintendo')
df['Platform'] = df.Platform.replace('WiiU','Nintendo')
df['Platform'] = df.Platform.replace('NES','Nintendo')
df['Platform'] = df.Platform.replace('Gameboy','Nintendo')
df['Platform'] = df.Platform.replace('N64','Nintendo')
df['Platform'] = df.Platform.replace('SCD','Nintendo')
df['Platform'] = df.Platform.replace('GBA','Nintendo')

# Computer
df['Platform'] = df.Platform.replace('PC','Computer')
df['Platform'] = df.Platform.replace('PSV','Computer')

# Sega
df['Platform'] = df.Platform.replace('SAT','Sega')
df['Platform'] = df.Platform.replace('DC','Sega')
df['Platform'] = df.Platform.replace('GEN','Sega')
df['Platform'] = df.Platform.replace('GG','Sega')

# Other
df['Platform'] = df.Platform.replace(2600,'Other')
df['Platform'] = df.Platform.replace('NG','Other')
df['Platform'] = df.Platform.replace('WS','Other')
df['Platform'] = df.Platform.replace('3DO','Other')
df['Platform'] = df.Platform.replace('TG16','Other')
df['Platform'] = df.Platform.replace('PCFX','Other')

In [ ]:
df.Platform.value_counts()

In [ ]:
# create dummy variables and add them to the df
dummy_df =  pd.get_dummies(df['Platform'])
dummy_df.columns = ['Nintendo', 'Playstation', 'Xbox', 'Computer', 
                    'Sega', 'Other']
df = pd.concat([df, dummy_df], axis=1)

In [ ]:
df.head()

In [ ]:
df.Genre.value_counts()

In [ ]:
df['Genre'] = df.Genre.replace('Puzzle','Strategy')
df['Genre'] = df.Genre.replace('Action','Action-Adventure')
df['Genre'] = df.Genre.replace('Adventure','Action-Adventure')
df['Genre'] = df.Genre.replace('Fighting','Simulation')
df['Genre'] = df.Genre.replace('Racing','Simulation')

In [ ]:
df.Genre.value_counts()

In [ ]:
# create dummy variables and add them to the df
dummy_df =  pd.get_dummies(df['Genre'])
dummy_df.columns = ['Action-Adventure', 'Simulation', 'Sports', 
                    'Misc', 'Role-Playing', 'Shooter', 
                    'Strategy', 'Platform']
df = pd.concat([df, dummy_df], axis=1)

In [ ]:
df.head()

In [ ]:
#drop features
df = df.drop(['Publisher', 'Platform', 'Genre'], axis=1)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.Global_Sales.unique()

# Lets see the distribution

In [ ]:
sns.distplot(df['Global_Sales'])

In [ ]:
sns.distplot(df['Year'])

In [ ]:
sns.distplot(df['Global_Sales'])

In [ ]:
sns.distplot(df['Nintendo'])

In [ ]:
df.info()

In [ ]:
train_validate, test = train_test_split(df, test_size=.2, random_state=1234)
train, validate = train_test_split(train_validate, test_size=.3, 
                                       random_state=1234)

In [ ]:
train.head()

In [ ]:
validate.head()

In [ ]:
test.head()

In [ ]:
X_train = train.drop(columns = ['Global_Sales'])
y_train = pd.DataFrame(train.Global_Sales)
X_validate = validate.drop(columns=['Global_Sales'])
y_validate = pd.DataFrame(validate.Global_Sales)
X_test = test.drop(columns=['Global_Sales'])
y_test = pd.DataFrame(test.Global_Sales)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
X_validate.head()

In [ ]:
y_validate.head()

In [ ]:
X_test.head()

In [ ]:
y_test.head()

In [ ]:
scale_columns = ['Global_Sales', 'Year']
scaler = MinMaxScaler()
scaler.fit(train[scale_columns])

train_scaled = scaler.transform(train[scale_columns])
validate_scaled = scaler.transform(validate[scale_columns])
test_scaled = scaler.transform(test[scale_columns])
#turn into dataframe
train_scaled = pd.DataFrame(train_scaled)
validate_scaled = pd.DataFrame(validate_scaled)
test_scaled = pd.DataFrame(test_scaled)

In [ ]:
train_scaled.head()

In [ ]:
validate_scaled.head()

In [ ]:
test_scaled.head()